<a href="https://colab.research.google.com/github/ctruciosm/BasicStats/blob/main/AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise Fatorial
By Carlos Trucíos

In [ ]:
install.packages("psych")
install.packages("mnt")
install.packages("POET")

In [4]:
library(psych)
library(mnt)
library(dplyr)
library(POET)

# 0. Dados

Utilizaremos o [_data set_](https://raw.githubusercontent.com/ctruciosm/ctruciosm.github.io/master/datasets/monthly_data.csv) `monthly_data.csv` que contém os retornos mensais de 16 ações negociadas na bolsa e valores de São Paulo entre Janeiro 2000 até Fevereiro 2022.

In [5]:
# Importar dados
dados = read.csv("https://raw.githubusercontent.com/ctruciosm/ctruciosm.github.io/master/datasets/monthly_data.csv")
head(dados)
dados = scale(as.matrix(dados), center = TRUE, scale = FALSE)

,ALPA4,BBAS3,BBDC3,BBDC4,CMIG4,CPLE6,ELET3,ELET6,EMBR3,GGBR4,ITSA4,LIGT3,PETR3,PETR4,SBSP3,VIVT3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-6.35512370,-5.952382,-2.078679,3.200547,-23.950618,-17.714287,-17.047656,-13.722833,-7.888889,4.166667,-8.500541,20.628142,-14.991126,-11.304351,-5.7048544,33.243479
2,2.42718215,-5.696199,-10.708152,-12.924411,-5.487014,12.500005,-4.392524,-7.173605,1.745443,-1.600005,-3.529415,1.641325,26.466875,12.745102,-12.2499986,18.749998
3,9.95260583,-14.248766,9.934533,13.243936,4.706286,-6.507126,1.010101,3.554871,8.795633,-7.520324,3.844540,-9.832372,4.304947,4.449614,0.5697932,-15.444107
4,-5.11890398,-4.610491,-6.916793,-8.119986,-7.685016,-9.084745,-13.870968,-11.044779,6.145254,-4.175821,-8.823535,-5.299998,-9.090907,-8.877120,-3.3982805,-11.151319
5,14.81339362,-4.999999,-4.789383,-6.572207,2.181821,9.619687,8.913859,3.691275,-13.789478,-4.862384,-3.870964,-12.642799,1.249999,-2.580794,-21.9882677,2.184377
6,0.01666724,13.157897,20.656147,27.561080,14.313816,15.646263,27.235216,29.126219,34.736059,8.230900,17.650236,12.631870,30.814813,30.071599,10.5225306,-28.623187


In [6]:
test.BHEP(dados, MC.rep = 1000)

  |======================================================================| 100%



------------------------------------------------------------------------- 

         Test for multivariate normality with the BHEP  teststatistic.

tuning parameter = 1  
BHEP  =  1.10135  
critical value =   1.000175  (via monte carlo) 


------------------------------------------------------------------------- 

## 1. É pertinente aplicar AF?

- Teste de esfericidade de Barlett: $$H_0: \textbf{R} = \textbf{I} \quad vs. \quad H_1: \textbf{R} \neq = \textbf{I}$$ 

Estaística de teste: $-[n - 1 -(2p+5)/6] \log |\textbf{R}| \sim \chi^2_{(p^2-p)/2}$

- Medida de adequação amostral de Kaisser, Meyer e Olkin (**KMO**): 


$$KMO = MSA = \dfrac{\displaystyle \sum \sum_{h \neq j} r_{hj}^2}{\displaystyle \sum \sum_{h \neq j} r_{hj}^2 + \displaystyle \sum \sum_{h \neq j} a_{hj}^2},$$
em que $r_{hj}$ é o coeficiente de correlação entre $X_h$ e $X_j$ e $a_{hj}$ é o coeficiente de correlação parcial entre as variáveis. 

Se o mdelo fatorial pode ser aplicado, $a_{hj}$ será zero ou muito pequeno (pois "retirar"o efeito da outra variáve equivale a retirar o efeito dos fatores comuns), fazendo com que KMO seja próximo de 1 (valores inferiores a 0.5 não são desejáveis).

Baseado no KMO, pode també ser calculado a medida de adequação amostral para cada uma das variáveis.

$$MSA_j = \dfrac{\displaystyle \sum_{h \neq j} r_{hj}^2}{\displaystyle \sum_{h \neq j} r_{hj}^2 + \displaystyle \sum_{h \neq j} a_{hj}^2},$$


In [7]:
cortest.bartlett(cor(dados), n = nrow(dados), diag = TRUE) # Requer normalidade

$chisq
[1] 3614.428

$p.value
[1] 0

$df
[1] 120

In [8]:
KMO(dados)

Kaiser-Meyer-Olkin factor adequacy
Call: KMO(r = dados)
Overall MSA =  0.88
MSA for each item = 
ALPA4 BBAS3 BBDC3 BBDC4 CMIG4 CPLE6 ELET3 ELET6 EMBR3 GGBR4 ITSA4 LIGT3 PETR3 
 0.95  0.96  0.87  0.84  0.92  0.93  0.80  0.79  0.87  0.95  0.93  0.95  0.78 
PETR4 SBSP3 VIVT3 
 0.79  0.96  0.92 

## 2. Aplicar AF

- Covariância ou Correlação? (seguir os mesmos critérios do ACP)
- Quantos fatores?
- Qual método de estimação utilizar?
- Qual método para extrair os fatores?

In [9]:
apply(dados, 2, sd)

ALPA4     BBAS3     BBDC3     BBDC4     CMIG4     CPLE6     ELET3     ELET6 
10.971053 12.029736  9.634486  9.780371 10.613703 10.000633 14.603155 13.046453 
    EMBR3     GGBR4     ITSA4     LIGT3     PETR3     PETR4     SBSP3     VIVT3 
11.641115 13.110303  8.631587 13.487736 12.062489 11.996239 10.517618  7.673349

In [60]:
POETKhat(t(dados))

5.036303,5.215364,5.394690,5.574049,5.753488,5.932980,6.112495,6.292081,6.471704,6.651370
5.040173,5.223104,5.406301,5.589530,5.772840,5.956201,6.139586,6.323043,6.506536,6.690072


Segundo o visto na aula, a matriz de cargas fatoriais e os fatores são obtidos da seguinte forma:

$$\hat{\textbf{F}} = \textbf{x} \textbf{P}_1\textbf{D}_1^{-1/2} \quad e \quad  \hat{\Lambda} = \textbf{P}_1 \textbf{D}_1^{1/2}$$


In [10]:
# Manual
dados = scale(dados, scale = TRUE)
eigen_aux <- eigen(cor(dados))
P <- eigen_aux$vectors       
D <- diag(eigen_aux$values) 
F_hat <- dados %*% P[,1] * 1/sqrt(D[1, 1])
Lambda_hat <- P[, 1] * sqrt(D[1, 1])

In [27]:
modelo_af_pca = principal(dados, nfactors = 1, rotate = "none", scores = TRUE)

In [28]:
# As cargas fatoriais feitas na mao e com a implementação batem?
cbind(Lambda_hat, modelo_af_pca$loadings)

,Lambda_hat,PC1
ALPA4,-0.4723331,0.4723331
BBAS3,-0.8230853,0.8230853
BBDC3,-0.8025489,0.8025489
BBDC4,-0.8353218,0.8353218
CMIG4,-0.7525191,0.7525191
CPLE6,-0.7153460,0.7153460
ELET3,-0.7002974,0.7002974
ELET6,-0.6931126,0.6931126
EMBR3,-0.3303562,0.3303562
GGBR4,-0.6789408,0.6789408


In [29]:
# Os fatores (scores) feitos na mão e com a implementação batem?
head(cbind(F_hat, modelo_af_pca$scores))

,,PC1
1,0.89050095,-0.89050095
2,0.19590467,-0.19590467
3,0.07818002,-0.07818002
4,1.17197970,-1.17197970
5,0.48120457,-0.48120457
6,-1.94134826,1.94134826


In [31]:
# Rotação
## Como temos k = 1, não faz sentido fazermos rotação, mas para ilustrar, faremos k = 2
modelo_af_pca_sem_rotar = principal(dados, nfactors = 2, scores = TRUE, rotate = "none")
modelo_af_pca_varimax = principal(dados, nfactors = 2, scores = TRUE, rotate = 'varimax')

In [32]:
modelo_af_pca_sem_rotar$loadings


Loadings:
      PC1    PC2   
ALPA4  0.472       
BBAS3  0.823 -0.179
BBDC3  0.803 -0.308
BBDC4  0.835 -0.307
CMIG4  0.753  0.209
CPLE6  0.715  0.290
ELET3  0.700  0.579
ELET6  0.693  0.584
EMBR3  0.330 -0.209
GGBR4  0.679       
ITSA4  0.848 -0.229
LIGT3  0.601  0.233
PETR3  0.756 -0.257
PETR4  0.801 -0.250
SBSP3  0.616  0.136
VIVT3  0.425       

                 PC1   PC2
SS loadings    7.715 1.335
Proportion Var 0.482 0.083
Cumulative Var 0.482 0.566

In [36]:
modelo_af_pca_varimax$loading


Loadings:
      RC1   RC2  
ALPA4 0.402 0.254
BBAS3 0.750 0.384
BBDC3 0.815 0.272
BBDC4 0.841 0.293
CMIG4 0.449 0.639
CPLE6 0.369 0.678
ELET3 0.174 0.892
ELET6 0.165 0.892
EMBR3 0.388      
GGBR4 0.582 0.361
ITSA4 0.801 0.361
LIGT3 0.317 0.562
PETR3 0.747 0.281
PETR4 0.777 0.315
SBSP3 0.390 0.496
VIVT3 0.330 0.267

                 RC1   RC2
SS loadings    5.145 3.904
Proportion Var 0.322 0.244
Cumulative Var 0.322 0.566

In [37]:
# Outros métodos de estimar as cargas/fatores
modelo_af_fp = fa(dados, nfactors = 1, fm = "pa")
modelo_af_mv = fa(dados, nfactors = 1, fm = "ml")

In [38]:
cbind(modelo_af_pca$loadings, modelo_af_fp$loadings, modelo_af_mv$loading)

,PC1,PA1,ML1
ALPA4,0.4723331,0.4356876,0.4161115
BBAS3,0.8230853,0.8133646,0.8168817
BBDC3,0.8025489,0.7903095,0.8712896
BBDC4,0.8353218,0.8295515,0.9040527
CMIG4,0.7525191,0.7289212,0.6887567
CPLE6,0.7153460,0.6870637,0.6428744
ELET3,0.7002974,0.6699717,0.6019735
ELET6,0.6931126,0.6621171,0.5942171
EMBR3,0.3303562,0.3004060,0.3019552
GGBR4,0.6789408,0.6476274,0.6138870


In [39]:
head(cbind(modelo_af_pca$scores, modelo_af_fp$scores, modelo_af_mv$scores))

,PC1,PA1,ML1
1,-0.89050095,-0.92646362,-0.8069548
2,-0.19590467,-0.39642738,-0.5908011
3,-0.07818002,-0.02051454,0.2440422
4,-1.17197970,-1.13289355,-1.1216091
5,-0.48120457,-0.52742879,-0.5960306
6,1.94134826,2.01540279,2.1362188


Mais detalhes dos outputs podem ser encontrados [aqui](https://m-clark.github.io/posts/2020-04-10-psych-explained/)